In [1]:
#import packages
import pandas as pd
import math
import statsmodels.stats.api as sms
import scipy.stats as st

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
#import data
raw_data = pd.read_csv("ab_data.csv")
df = raw_data.copy()

print("Number of rows: ", df.shape[0], " Number of columns: ", df.shape[1])
df.head()

Number of rows:  294478  Number of columns:  5


,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [7]:
#some of the control group saw the new_page and some tretment group saw the old_page - delete these instances
mask1 = (df["group"] == "control") & (df["landing_page"] == "new_page")
index_to_drop1 = df[mask1].index
df = df.drop(index_to_drop1)

mask2 = (df["group"] == "treatment") & (df["landing_page"] == "old_page")
index_to_drop2 = df[mask2].index
df = df.drop(index_to_drop2)


#drop duplicated users
df.drop_duplicates(subset ='user_id',keep ='first',inplace = True)

In [8]:
df['group'].value_counts()

treatment    145310
control      145274
Name: group, dtype: int64

In [11]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [30]:
df_pivot = df.pivot_table(index='group', columns='converted', values='user_id', aggfunc='count')
df_pivot['convert_pct'] = df_pivot[1]*1.00/(df_pivot[1]+df_pivot[0])
df_pivot['total'] = (df_pivot[1]+df_pivot[0])
df_pivot

converted,0,1,convert_pct,total
group,,,,
control,127785,17489,0.120386,145274
treatment,128046,17264,0.118808,145310


In [40]:
import statsmodels.stats.proportion as ssp
z_score, p_value = ssp.proportions_ztest(count=df_pivot[1],
                                         nobs=df_pivot.total, 
                                         alternative="smaller")
print("Z值为：", z_score)
print("P值为：", p_value)

Z值为： 1.3109241984234394
P值为： 0.9050583127590245


Result: Do not reject the null hypothesis

In [42]:
# https://github.com/RobbieGeoghegan/AB_Testing/blob/master/AB_Testing.ipynb

#Calculate pooled standard error and margin of error
se_pooled = math.sqrt(prob_pooled * (1 - prob_pooled) * (1 / total_users_control + 1 / total_users_treatment))
z_score = st.norm.ppf(1 - confidence_level / 2)
margin_of_error = se_pooled * z_score

#Calculate dhat, the estimated difference between probability of conversions in the experiment and control groups
d_hat = (conversions_treatment / total_users_treatment) - (conversions_control / total_users_control)

#Test if we can reject the null hypothesis
lower_bound = d_hat - margin_of_error
upper_bound = d_hat + margin_of_error

if practical_significance < lower_bound:
    print("Reject null hypothesis")
else: 
    print("Do not reject the null hypothesis")
    
print("The lower bound of the confidence interval is ", round(lower_bound * 100, 2), "%")
print("The upper bound of the confidence interval is ", round(upper_bound * 100, 2), "%")

Do not reject the null hypothesis
The lower bound of the confidence interval is  -0.39 %
The upper bound of the confidence interval is  0.08 %
